In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import wandb
import functools

from src.display import showarray
from src.mask import MaskGenerator
from src.datagen import DatasetFillGenerator
from src.augmenters import masked_channel_augmenter
from src.builders.unet import UNETBuilder
from src.loss import MaskedMAE, MaskedGaussedSobelMAE, GaussedSobelMAE, CombinedLoss

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
config = {
  "learning_rate": 0.001,
  "epochs": 16,
  "batch_size": 16,
  "mask_gen_degree": "HEAVY",
  "mask_gen_min_width": 5,
  "mask_gen_max_width": 12,
  "learning_rate": 0.001
}
model_params = {
  "n_filters": 32,
  "n_layers": 3,
  "n_convs": 1,
  "activation": "elu",
  "dropout_rate": 0.4,
}
loss_dict = {
  "masked_mae": MaskedMAE(),
  "masked_gaussed_sobel_mae": MaskedGaussedSobelMAE(),
  "gaussed_sobel_mae": GaussedSobelMAE(),
  "mae": tf.keras.losses.MeanAbsoluteError(),
}
loss_weights = {
  "masked_mae": 1.0,
  "masked_gaussed_sobel_mae": 1.0,
  "gaussed_sobel_mae": 1.0,
  "mae": 1.0,
}

loss_config = {key: (loss_fn, loss_weights[key]) for key, loss_fn in loss_dict.items()}

config.update({f"unet_{key}": val for key, val in model_params.items()})
config.update({f"loss_{key}_weight": val for key, val in loss_weights.items()})
wandb.init(project="cv3-ii-ae-unet", entity="put_dl_team", config=config)

IMAGE_SIZE = (256, 256)
CHANNELS = 4
BATCH_SIZE = wandb.config["batch_size"]

MASK_GEN_PARAM = {
    "degree": wandb.config["mask_gen_degree"],
    "min_width": wandb.config["mask_gen_min_width"],
    "max_width": wandb.config["mask_gen_max_width"],
}

mask_generator = MaskGenerator(*IMAGE_SIZE, CHANNELS-1, **MASK_GEN_PARAM)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sbartekt (put_dl_team). Use `wandb login --relogin` to force relogin


In [4]:
def scale(tensor: tf.Tensor, divisor: float = 255.0) -> tf.Tensor:
    return tensor / divisor

def recast_to_image(tensor: tf.Tensor) -> np.ndarray:
    return tf.cast(tensor[:, :, :3] * 255, tf.uint8).numpy()


ds_train, ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    directory="../data/1-8size", label_mode=None, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
    shuffle=True, seed=42, validation_split=0.1, subset="both",
)
ds_train = ds_train.map(scale)
ds_valid = ds_valid.map(scale)

for batch in ds_valid.take(1):
    showcase_images = batch[:5]


class ImageFillCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, showcase_images, mask_generator):
        self.model = model
        self.masked_images, self.showcase_images = masked_channel_augmenter(showcase_images, mask_generator)

    def on_epoch_end(self, epoch, logs=None):
        nn_filled = self.model.predict(self.masked_images)
        all_joint = []
        for i in range(5):
            masked = recast_to_image(self.masked_images[i])
            original_image = recast_to_image(self.showcase_images[i])
            filled_image = recast_to_image(nn_filled[i])
            joint = np.concatenate([masked, original_image, filled_image], axis=1)
            all_joint.append(joint)
        all_joint = np.concatenate(all_joint, axis=0)
        wandb.log({"sample_fill": wandb.Image(all_joint)})

dataset_image_augmenter = functools.partial(masked_channel_augmenter, mask_generator=mask_generator)

np.random.seed(42)
train_generator = DatasetFillGenerator(ds_train, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)
valid_generator = DatasetFillGenerator(ds_valid, IMAGE_SIZE, CHANNELS, dataset_image_augmenter)

Found 2188 files belonging to 1 classes.
Using 1970 files for training.
Using 218 files for validation.


In [5]:
IM_SHAPE = (*IMAGE_SIZE, CHANNELS)
builder = UNETBuilder(IM_SHAPE, IM_SHAPE, **model_params)
model = builder.build()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  1184        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 128, 128, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [6]:
loss = CombinedLoss(loss_config)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config["learning_rate"]),
    loss=loss,
)

In [7]:
model.fit(train_generator, epochs=wandb.config["epochs"], validation_data=valid_generator, callbacks=[wandb.keras.WandbCallback(), ImageFillCallback(model, showcase_images, mask_generator)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/16
124/124 [==============================] - ETA: 0s - loss: 0.2219

INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230122_125438-nbaaqp3a\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230122_125438-nbaaqp3a\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\Bartosz\PycharmProjects\CV3_PROJECT\cv-image-inpainting\notebooks\wandb\run-20230122_125438-nbaaqp3a\files\model-best)... Done. 0.1s


ValueError: in user code:

    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Bartosz\.conda\envs\deep\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_input_compat

In [ ]:
wandb.finish(0)